In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [5]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df[ 'prices_amountmin' ].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [11]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [14]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
# lowercase (map() was used as a result of observation of the data: df.head() )

run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [15]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [16]:
# Feature engineering


df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  
  return output_dict


df['features_parsed'] = df['features'].map(parse_features)

In [18]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [19]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

476

In [20]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  # print(key)
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [21]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_jacket length', 'feat_footwear type', 'feat_smart watch',
       'feat_waterproof', 'feat_package', 'feat_adidas',
       'feat_contains cubic zirconia', 'feat_purpose',
       'feat_looks best on these face shapes:', 'feat_jewelry setting'],
      dtype='object', length=526)

In [25]:
df[ df['feat_jacket length'].isnull() ].shape # how many null rows for this column?

(18272, 526)

In [23]:
df.shape[0]

18280

In [27]:
df[ False == df['feat_jacket length'].isnull() ].shape[0] # how many valuable rows for this column?

8

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100
# keys_stat

In [30]:
{k: v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]


for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
  

In [34]:
df[ df.brand != df.feat_brand ].shape

(18228, 531)

In [36]:
df[ df.brand != df.feat_brand ][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [38]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand != df.feat_brand ][ ['brand', 'feat_brand']].head()



,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [39]:
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = []

In [41]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [44]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feats, model)

(-57.172328503725836, 4.227341275008907)

In [45]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat


In [46]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feats, model)

(-57.12896870527643, 4.1608099520676705)

In [47]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feats, model)

(-57.20148875865986, 4.149725476371453)

In [49]:
df['brand'].value_counts(normalize=True)

nike                           0.097210
puma                           0.033315
ralph lauren                   0.028775
vans                           0.021116
new balance                    0.020295
                                 ...   
big smith                      0.000055
avia peter                     0.000055
bobux                          0.000055
tony sports accessories 888    0.000055
gelato mens                    0.000055
Name: brand, Length: 1732, dtype: float64

In [50]:
df[ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [53]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feats, model)

(-57.174254322129265, 4.237153476213779)

In [55]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'sport': 'any sport', 'style': 'pants', 'condition': 'new with tags'},
       {'primary color': 'blue', 'battery type': 'does not contain a battery', 'lens tint': 'grey', 'product in inches (l x w x h)': '6.0 x 2.0 x 2.0', 'assembled product dimensions (l x w x h)': '6.00 x 2.00 x 2.00 inches', 'is portable': 'y', 'frame material': 'nylon', 'model': 'ev0800-400', 'uv rating': '400', 'multi pack indicator': 'no', 'gender': 'men', 'brand': 'nike', 'age group': 'adult', 'lens type': 'sunglasses', 'features': 'wrap around fit; water repellent lenses; interchangeable lenses', 'material': 'plastic', 'model no.': 'ev0800-400', 'shape': 'semi-rimless', 'eyewear frame style': 'semi-rimless', 'shipping weight (in pounds)': '1.0', 'walmart no.': '554240966', 'manufacturer part number': 'ev0800-400'},
       {'name': 'jordan true flight (box only)'}, {},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'allumium/white/persian violet', 'casual & dress shoe style': 'athletic 

In [56]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
infant                 4
mens                   4
toddler                4
boys'                  3
youth                  2
women ,�� unisex       2
women                  2
men||women             2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

# after adding for-loop to generate new '*_cat' categories above 

In [0]:
# feats_cat = [x for x in df.columns if 'cat' in]
feats_cat = [x for x in df.columns if x.endswith('_cat')]
# feats_cat
feats += feats_cat
feats = list(set(feats))

In [64]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(feats, model)

(-57.672792443234144, 4.176243757854901)

In [65]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2419 ± 0.0107,brand_cat
0.1007 ± 0.0125,feat_material_cat
0.0126 ± 0.0021,feat_weight_cat
0.0123 ± 0.0019,feat_brand_cat
0.0105 ± 0.0028,feat_adjustable_cat
0.0096 ± 0.0006,feat_fabric content_cat
0.0055 ± 0.0005,feat_shoe category_cat
0.0053 ± 0.0017,feat_resizable_cat
0.0044 ± 0.0010,feat_color_cat
0.0043 ± 0.0001,feat_fabric material_cat


In [91]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_weight_cat', 'feat_material_cat', 'feat_style_cat', 'feat_gender_cat', 'feat_color_cat', 'feat_fabric content_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
result = run_model(feats, model)
print(result)

(-56.95433804759075, 4.215747910807833)


In [92]:
X = df[ feats ].values
y = df[ 'prices_amountmin' ].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)
print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names=feats)

(-56.95433804759075, 4.215747910807833)


Weight,Feature
0.2512 ± 0.0075,brand_cat
0.1017 ± 0.0124,feat_material_cat
0.0456 ± 0.0043,feat_gender_cat
0.0252 ± 0.0007,feat_brand_cat
0.0166 ± 0.0014,feat_fabric content_cat
0.0131 ± 0.0012,feat_weight_cat
0.0087 ± 0.0016,feat_color_cat
0.0068 ± 0.0017,feat_style_cat


In [0]:
!git add matrix_one/day4.ipynb